First we import all the needed modules. Your fbx.pyd should be in your installations site-packages directory. Generating the fbx.pyd is unfortunately very platform specifc and I can't help you with that.

In [116]:
import sys
import os
import math
import fbx
import json

Now we fix up the path to include the FbxCommon module.

In [2]:
sys.path = sys.path + [os.curdir + os.sep + "fbx2020_1"]    

And we need the euclid module for math

In [215]:
sys.path = sys.path + [os.curdir + os.sep + "pyeuclid"]

In [220]:
from euclid import Point3
from FbxCommon import *

Next we create the global objects we need to work with : the FbxManager and the FbxScene

In [7]:
def create_sdk_manager():
    result = FbxManager.Create()
    return result

def create_scene(manager, name = "FbxScene"):
    ios = FbxIOSettings.Create(manager, IOSROOT)
    manager.SetIOSettings(ios)
    scene = FbxScene.Create(manager, name)
    return scene

sdk_manager = create_sdk_manager()
fbx_scene = create_scene(sdk_manager)

Next, we load the secne, specifiying which elements we want to import, which in our case is all of them.

In [15]:
def load_scene(manager, scene, file_name):
    importer = FbxImporter.Create(manager, "")    
    result = importer.Initialize(file_name, -1, manager.GetIOSettings())
    if not result:
        return False    
    if importer.IsFBX():
        manager.GetIOSettings().SetBoolProp(EXP_FBX_MATERIAL, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_TEXTURE, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_EMBEDDED, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_SHAPE, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_GOBO, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_ANIMATION, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_GLOBAL_SETTINGS, True)
    
    result = importer.Import(scene)
    importer.Destroy()
    return scene

if load_scene(sdk_manager, fbx_scene, "GenericMan.fbx") == None:
    print("Load Failed")

The scene is organsied as a tree. So we want the root node to begin traversal

In [16]:
 root_node = fbx_scene.GetRootNode()

Now we can walk the tree with the usual recursive jiggery pokey

In [128]:
def walk_nodes_aux(node):
    print(node.GetName())
    for i in range(node.GetChildCount()):
        walk_nodes_aux(node.GetChild(i))
        
def walk_nodes(scene):
    root = scene.GetRootNode()
    walk_nodes_aux(root)
    
walk_nodes(fbx_scene)

RootNode
Character
Root
Waist
Left Hip
Left Knee
Left Foot
Right Hip
Right Knee
Right Foot
Middle Spine
Neck
Left Arm
Left Elbow
Left Hand
Head
Right Arm
Right Elbow
Right Hand


Each node has a transform that is multiplied through the hierarchy and varies over time, with animation. 

In [129]:
def get_node_global_transform(node, time = 0.0):
    t = FbxTime(time)
    return node.EvaluateGlobalTransform(t)

In [130]:
def get_node_local_transform(node, time = 0.0):
    t = FbxTime(time)
    return node.EvaluateLocalTransform(t)

And also, a helper transform, which does not form part of the transform hierarchy which provices an "in-place" offset.

In [139]:
def get_node_geometric_transform(node):
    result = fbx.FbxAMatrix()
    result.SetIdentity()
    if (node and node.GetNodeAttribute()):
        t = node.GetGeometricTranslation(fbx.FbxNode.eSourcePivot)
        r = node.GetGeometricRotation(fbx.FbxNode.eSourcePivot)
        s = node.GetGeometricScaling(fbx.FbxNode.eSourcePivot)
        result = fbx.FbxAMatrix(t,r,s)
    return result

Taken all together, they can calculate the nodes transform.

In [108]:
def get_node_transform(node, time = 0.0):
    result = get_node_global_transform(node, time)
    if (node.GetNodeAttribute()):
        gt = get_node_geometric_transform(node)
        result = result * gt
    return result

In [109]:
transform = get_node_transform(root_node, 0.0)
print(transform)

Not a very useful representation. So we need to write a function to print it.

In [110]:
def print_matrix(m):
    for i in range(0,3):
        print("|", m[i][0], ",", m[i][1], ",", m[i][2], ",", m[i][3], "|") 
    return

print_matrix(transform)

| 1.0 , 0.0 , 0.0 , 0.0 |
| 0.0 , 1.0 , 0.0 , 0.0 |
| 0.0 , 0.0 , 1.0 , 0.0 |


We can work out a node local transform if we have the global transform of parent and child.

In [97]:
def get_node_local_transform(node, time = 0.0):
    transform = get_node_transform(node, time)
    parent = node.GetParent()
    if parent:
        parent_transform = get_node_transform(parent, time)
        inverse_parent_transform = parent_transform.Inverse()
        transform = transform * inverse_parent_transform
    return transform

In [98]:
local_transform = get_node_local_transform(root_node, 0.0)
print_matrix(local_transform)

| 1.0 , 0.0 , 0.0 , 0.0 |
| 0.0 , 1.0 , 0.0 , 0.0 |
| 0.0 , 0.0 , 1.0 , 0.0 |


Thats not so useful, so let's look at one of the childern

In [99]:
child_node = root_node.GetChild(1)
child_transform = get_node_local_transform(child_node, 0.0)
print_matrix(child_transform)

| 0.0 , 0.0 , 0.0 , 0.0 |
| 0.0 , 0.0 , 0.0 , 0.0 |
| 0.0 , 0.0 , 0.0 , 0.0 |


Each node has an attribute which describes which type of data is associated with it

In [119]:
print(root_node.GetNodeAttribute())
print(child_node.GetNodeAttribute())
print(root_node.GetChild(0).GetNodeAttribute())

None


Every fbx type has a classId which we can use to identify it

In [158]:
fbx.FbxMesh.ClassId.GetName()
print(child_node.GetNodeAttribute().ClassId.GetName())

FbxSkeleton


So now we can dump a lot more useful information for each node, and we can have a custom handler function for each node. This is the backbone of an exporter.

In [159]:
def process_mesh(node, time = 0.0):
    print("It's a mesh!")
    return

def process_skeleton(node, time = 0.0):
    print("It's a skeleton")
    return
    
fbx_node_methods = {}

fbx_node_methods[fbx.FbxMesh.ClassId.GetName()] = process_mesh
fbx_node_methods[fbx.FbxSkeleton.ClassId.GetName()] = process_skeleton 

def walk_nodes_aux(node, time = 0.0):
    print(node.GetName())
    node_attribute = node.GetNodeAttribute()
    global_transform = get_node_global_transform(node, time)
    geometric_transform = get_node_geometric_transform(node)
    actual_transform = global_transform * geometric_transform
    print_matrix(actual_transform)
    if (node_attribute and node_attribute.ClassId.GetName() in fbx_node_methods):
        fbx_node_methods[node_attribute.ClassId.GetName()](node, time)
    else:
        print("Unknown node type")
    for i in range(node.GetChildCount()):
        walk_nodes_aux(node.GetChild(i))
        
walk_nodes_aux(root_node)

RootNode
| 1.0 , 0.0 , 0.0 , 0.0 |
| 0.0 , 1.0 , 0.0 , 0.0 |
| 0.0 , 0.0 , 1.0 , 0.0 |
Unknown node type
Character
| 1.0 , 0.0 , 0.0 , 0.0 |
| 0.0 , 1.0 , 0.0 , 0.0 |
| 0.0 , 0.0 , 1.0 , 0.0 |
It's a mesh!
Root
| 1.0 , 0.0 , 0.0 , 0.0 |
| 0.0 , 1.0 , 0.0 , 0.0 |
| 0.0 , 0.0 , 1.0 , 0.0 |
It's a skeleton
Waist
| 1.0 , 0.0 , 0.0 , 0.0 |
| 0.0 , 0.978147720798726 , 0.2079116993217334 , 0.0 |
| 0.0 , -0.20791162496672436 , 0.9781473709858831 , 0.0 |
It's a skeleton
Left Hip
| 1.0000331401824951 , 0.0 , 0.0 , 0.0 |
| 0.0 , 0.6427872521177278 , -0.7660440428796939 , 0.0 |
| 0.0 , 0.7660587909941312 , 0.6427996272546944 , 0.0 |
It's a skeleton
Left Knee
| 1.0000349283810976 , 0.0 , 0.0 , 0.0 |
| 0.0 , 0.5150381899289473 , 0.8571677183484778 , 0.0 |
| 0.0 , -0.8571845273564864 , 0.515048289797239 , 0.0 |
It's a skeleton
Left Foot
| 1.0000349283810976 , 0.0 , 0.0 , 0.0 |
| 0.0 , 0.5150381899289473 , 0.8571677183484778 , 0.0 |
| 0.0 , -0.8571845273564864 , 0.515048289797239 , 0.0 |
It's a skelet

Now we need to process a mesh. A mesh is organised as a number of layers, each carrying different information.

In [167]:
test_mesh_node = root_node.GetChild(0)
test_mesh = test_mesh_node.GetNodeAttribute()

def parse_material(material):
    if material:
        if (material.GetReferenceMode() == FbxLayerElement.eDirect):
            print("Material is directly indexed")
        else:
            print("Material is indirectly indexed")
            index_array = material.GetIndexArray()
            index_array_count = index_array.GetCount()
            print("There are %d indices" % index_array_count)
    else:
        print("No material")
    
def parse_layers(node, mesh):
    print("Mesh has %d layers and %d materials " % (mesh.GetLayerCount(), node.GetMaterialCount()))
    material_count = node.GetMaterialCount()
    layer_count = mesh.GetLayerCount()
    for l in range(layer_count):
            print("In Layer %d " % (l))
            material = mesh.GetLayer(l).GetMaterials()
            parse_material(material)
            
parse_layers(test_mesh_node,test_mesh)
                

Mesh has 2 layers and 1 materials 
In Layer 0 
Material is indirectly indexed
There are 1300 indices
In Layer 1 
No material


But what are these access modes? When you have an array of values, like UV's for example - they can be stored that way - as an array. Or they can be stored as an array of values and an array of indices which you traverse to get the values. They can also be stored per vertex (ControlPoint) or per Polygon (Face) depending what they are: or ever per Vertex, per Polygon.

In practice these are the main sensible ways to store such an array, but there are a few others. This class addresses them, by abstracting away the differences and pretending everything is a per vertex, per poly attribute.


In [169]:
class LayerElement():
    def __init__(self, mesh, layer_index, layer_kind):
        self.mesh = mesh
        self.layer = mesh.GetLayer(layer_index)
        self.elements = self.layer.GetLayerElementOfType(layer_kind)
    def valid(self):
        return (self.layer) and (self.elements)
    def indexed(self):
        return (self.elements.GetReferenceMode() != FbxLayerElement.eDirect)
    def get_element(self, polygonIndex, vertexIndex):
        assert(self.mesh.GetPolygonCount() > polygonIndex)
        assert(self.mesh.GetPolygonSize() > vertexIndex)
        control_point_index = mesh.GetPolygonVertex(polygonIndex, vertexIndex)
        mapping_mode = self.elements.GetMappingMode()
        reference_mode = self.elements.GetReferenceMode() 
        if (mapping_mode == FbxLayerElement.eByControlPoint):
            if (reference_mode == FbxLayerElement.eDirect):
                return self.elements.GetDirectArray().GetAt(control_point_index)
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                index_array = elements.GetIndexArray()
                assert(index_array.GetCount() == mesh.GetControlPointsCount())
                assert(control_point_index < index_array.GetCount())
                ix = index_array.GetAt(control_point_index)
                direct_array = self.elements.GetDirectArray()
                assert(ix < direct_array.GetCount())
                return self.elements(ix)
        elif (mapping_mode == FbxLayerElement.eByPolygonVertex):
            vertexId = polygonIndex * 3 + vertexIndex;
            if (reference_mode == FbxLayerElement.eDirect):
                direct_array = self.elements.GetDirectArray()
                assert(vertexId < direct_array.GetCount())
                return direct_array.GetAt(vertexId)      
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                index_array = self.elements.GetIndexArray()
                assert(vertexId < index_array.GetCount())
                ix = index_array.GetAt(vertexId)
                return self.elements.GetDirectArray().GetAt(ix)
        elif (mapping_mode == FbxLayerElement.eByPolygon):
            if (reference_mode == FbxLayerElement.eDirect):
                direct_array = self.elements.GetDirectArray()
                assert(polygonIndex < direct_array.GetCount())
                return direct_array.GetAt(polygonIndex)
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                index_array = self.elements.GetIndexArray()
                assert(polygonIndex < index_array.GetCount())
                ix = index_array.GetAt(polygonIndex)
                return self.elements.GetDirectArray().GetAt(ix)
        elif (mapping_mode == FbxLayerElement.eAllSame):
            if (reference_mode == FbxLayerElement.eDirect):
                direct_array = self.elements.GetDirectArray()
                return direct_array.GetAt(0)
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                index_array = self.elements.GetIndexArray()
                ix = index_array.GetAt(0)
                return self.elements.GetDirectArray().GetAt(ix)
        else:
            print("Error - unhandled mapping mode!")
        return None           

A simpler helper function to get a per vertex attribute

In [170]:
    def get_vertex_element(self, vertexIndex):
        mapping_mode = self.elements.GetMappingMode()
        reference_mode = self.elements.GetReferenceMode()  
        if (mapping_mode == FbxLayerElement.eByControlPoint):
            if (reference_mode == FbxLayerElement.eDirect):
                return self.elements.GetDirectArray().GetAt(vertexIndex)
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                ix = self.elements.GetIndexArray().GetAt(vertexIndex)
                return self.elements.GetDirectArray().GetAt(ix)
        elif (mapping_mode == FbxLayerElement.eByPolygonVertex):
            pass
        elif (mapping_mode == FbxLayerElement.eByPolygon):
            pass
        elif (mapping_mode == FbxLayerElement.eByEdge):
            pass
        elif (mapping_mode == FbxLayerElement.eAllSame):
            return self.elements.GetDirectArray().GetAt(vertexIndex)
        return None

A simpler helper function to get a per-polygon attribute

In [171]:
   def get_polygon_element(self, polygonIndex):
        mapping_mode = self.elements.GetMappingMode()
        reference_mode = self.elements.GetReferenceMode()  
        if (mapping_mode == FbxLayerElement.eByControlPoint):
            if (reference_mode == FbxLayerElement.eDirect):
                return self.elements.GetDirectArray().GetAt(plygonIndex)
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                ix = self.elements.GetIndexArray().GetAt(polygonIndex)
                return self.elements.GetDirectArray().GetAt(ix)
        elif (mapping_mode == FbxLayerElement.eByPolygonVertex):
            pass
        elif (mapping_mode == FbxLayerElement.eByPolygon):
            if (reference_mode == FbxLayerElement.eDirect):
                return self.elements.GetDirectArray().GetAt(plygonIndex)
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                ix = self.elements.GetIndexArray().GetAt(polygonIndex)
                return self.elements.GetDirectArray().GetAt(ix)    
        elif (mapping_mode == FbxLayerElement.eByEdge):
            pass
        elif (mapping_mode == FbxLayerElement.eAllSame):
            if (reference_mode == FbxLayerElement.eDirect):
                direct_array = self.elements.GetDirectArray()
                return direct_array.GetAt(0)
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                index_array = self.elements.GetIndexArray()
                ix = index_array.GetAt(0)
                return self.elements.GetDirectArray().GetAt(ix)
        return None

Also a pair of helper functions to show us the mapping mode (element frequency) and reference mode (whether it's indexed or not)

In [194]:
def parse_mapping_mode(mode):
    mapping_types = [ "None", "By Control Point", "By Polygon and Control Point", "By Polygon", "By Edge", "All Same" ]
    return mapping_types[int(mode)]

In [195]:
def parse_reference_mode(mode):
    reference_types = [ "None", "Direct", "IndexToDirect"]
    return reference_types[int(mode)]

With all that out of the way, we can start examining the data on our mesh.

In [193]:
def parse_control_points(node, mesh):
    # vertices
    control_points_count = mesh.GetControlPointsCount()
    control_points = mesh.GetControlPoints()
    print(f"Mesh has {control_points_count} control points")
    layer_count = mesh.GetLayerCount()
    # normals
    for i in range(layer_count):
        print(f"Layer {i}")
        elements = LayerElement(mesh, i, FbxLayerElement.eNormal)
        normals = mesh.GetLayer(i).GetNormals()
        if normals:
            mapping_mode = normals.GetMappingMode()
            reference_mode = normals.GetReferenceMode()
            print("Reference mode %s" % parse_reference_mode(reference_mode))
            print("Mapping mode %s" % parse_mapping_mode(mapping_mode))    
        else:
            print("Layer %d has no normals" % (i))
 
parse_control_points(test_mesh_node, test_mesh)

Mesh has 1314 control points
Layer 0
Reference mode IndexToDirect
Mapping mode By Polygon Vertex
Layer 1
Layer 1 has no normals


Now we know how our data is structured, we can walk over it, poly by poly and vertex by vertex to get our data. We are going to assume the polys are triangles as, if they are not, the SDK always has a handy triangulate function.

In [218]:
print(sys.path)
from euclid import Point3 
v = test_mesh.GetControlPointAt(0)
vertex = Point3(v[0], v[1], v[2]) 

['D:\\Dev\\projects\\python\\pyglsl', 'd:\\apps\\python37\\python37.zip', 'd:\\apps\\python37\\DLLs', 'd:\\apps\\python37\\lib', 'd:\\apps\\python37', '', 'd:\\apps\\python37\\lib\\site-packages', 'd:\\apps\\python37\\lib\\site-packages\\win32', 'd:\\apps\\python37\\lib\\site-packages\\win32\\lib', 'd:\\apps\\python37\\lib\\site-packages\\Pythonwin', 'd:\\apps\\python37\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Zaba\\.ipython', '.\\fbx2020_1', '.\\pyeuclid']


In [228]:
from euclid import Point3


def process_mesh(node, mesh):
    vertices = []
    indices = []
    vertex_count = mesh.GetControlPointsCount()
    for vi in range(0, vertex_count):
        v = mesh.GetControlPointAt(vi)
        vertices += [ Point3(v[0], v[1], v[2]) ]
    poly_count = mesh.GetPolygonCount()
    for pi in range(0, poly_count):
        face_indices = []
        for vi in range(0, 3):
            i = mesh.GetPolygonVertex(pi, vi)
            face_indices += [ i ]
        indices += [ tuple(face_indices) ]
    return (vertices, indices)
            
process_mesh(test_mesh_node, test_mesh)

([Point3(-41.00, 57.40, 26.75),
  Point3(-41.00, 90.73, 26.75),
  Point3(0.00, 90.73, 25.69),
  Point3(0.00, 62.61, 14.79),
  Point3(-41.00, 124.07, 26.75),
  Point3(0.00, 124.07, 26.75),
  Point3(-41.00, 157.40, 26.75),
  Point3(0.00, 152.19, 18.39),
  Point3(14.98, 90.47, 24.39),
  Point3(9.58, 55.80, 17.83),
  Point3(15.61, 122.92, 24.98),
  Point3(14.25, 151.93, 17.48),
  Point3(36.66, 89.32, 11.90),
  Point3(35.70, 55.06, 11.60),
  Point3(41.11, 117.50, 13.83),
  Point3(30.80, 150.62, 13.42),
  Point3(35.70, 55.06, -11.60),
  Point3(36.66, 89.32, -11.90),
  Point3(14.98, 90.47, -24.39),
  Point3(9.58, 55.80, -17.83),
  Point3(41.11, 117.50, -13.83),
  Point3(15.61, 122.92, -24.98),
  Point3(30.80, 150.62, -13.42),
  Point3(14.25, 151.93, -17.48),
  Point3(0.00, 90.73, -25.69),
  Point3(0.00, 62.61, -14.79),
  Point3(0.00, 124.07, -26.75),
  Point3(0.00, 152.19, -18.39),
  Point3(-41.00, 90.73, -26.75),
  Point3(-41.00, 57.40, -26.75),
  Point3(-41.00, 124.07, -26.75),
  Point3(-41

Animations

In [115]:
def get_anim_stacks(scene):
    anim_stack_classid = FbxAnimStack.ClassId
    stack_names = fbx.FbxStringArray()
    scene.FillAnimStackNameArray(stack_names)
    num_stacks = stack_names.GetCount()
    for stack_index in range(0,num_stacks):
        anim_name = stack_names[stack_index]
        if anim_name == None:
            continue
        anim_stack = scene.FindMember(anim_stack_classid, anim_name.Buffer())
        reference_time_span = anim_stack.GetReferenceTimeSpan()
        local_time_span = anim_stack.GetLocalTimeSpan()
        print("Anim Stack ", anim_name)
        print("Reference Start, Stop ", anim_stack.GetReferenceTimeSpan().GetStart().GetSecondDouble(), anim_stack.GetReferenceTimeSpan().GetStop().GetSecondDouble())
        print("Local Start, Stop ", anim_stack.GetLocalTimeSpan().GetStart().GetSecondDouble(), anim_stack.GetLocalTimeSpan().GetStop().GetSecondDouble())
        
get_anim_stacks(fbx_scene)

Anim Stack  Idle
Reference Start, Stop  0.0 1.5000000779478457
Local Start, Stop  0.0 1.5000000779478457
Anim Stack  Walk
Reference Start, Stop  0.0 1.8000000921201813
Local Start, Stop  0.0 1.8000000921201813
Anim Stack  Jump
Reference Start, Stop  0.0 1.8000000921201813
Local Start, Stop  0.0 1.8000000921201813
Anim Stack  Crouch
Reference Start, Stop  0.0 1.8000000921201813
Local Start, Stop  0.0 1.8000000921201813


In [54]:
print FbxLight.ClassId

In [55]:
print root_node.GetChild(0).GetNodeAttribute().ClassId

In [96]:
def parse_materials(node, mesh):
    polygon_count = mesh.GetPolygonCount()
    layer_count = mesh.GetLayerCount()
    material = LayerElement(mesh, 0, FbxLayerElement.eMaterial)
    print dir(material)
    if (not(material.valid())):
        print "No materials"
        return None
    materials_used = []
    for i in range(polygon_count):
        material_connection_index = material.get_polygon_element(i) 
        if (not(material_connection_index in materials_used)):
            materials_used = materials_used + [ material_connection_index ]
    print "Mesh has %d materials" % (len(materials_used))
    uvset_names = []
    for l in range(layer_count):
        print "Layer %d " % (l)
        uvset_count = mesh.GetLayer(l).GetUVSetCount()
        print "Layer has %d uvsets " % (uvset_count)
        if (uvset_count != 0):
            uvsets = mesh.GetLayer(l).GetUVSets()
            for j in range(uvset_count):
                set_name = uvsets[j].GetName()
                print dir(set_name)
                print "Layer %d Set %d Name %s " % (l, j, set_name)
                if (not(set_name in uvset_names)):
                    uvset_names = uvset_names + [ set_name ]
    print materials_used
    print uvset_names
    return (materials_used, uvset_names)
                


In [81]:
def process_mesh(node):
    mesh = node.GetNodeAttribute()
    # print dir(mesh)
    mesh.GenerateNormals()
    mesh.GenerateTangentsDataForAllUVSets()
    print "Mesh has %d layers" % ( mesh.GetLayerCount() )
    polygon_count = mesh.GetPolygonCount()
    parse_materials(node,mesh)
    parse_control_points(node, mesh)
    parse_layers(node, mesh)

In [82]:
def process_bone(node):
    pass

In [83]:
def process_marker(node):
    pass

In [84]:
def classify_node(node):
    kinds = { FbxNodeAttribute.eMarker: "Marker",   FbxNodeAttribute.eSkeleton: "Bone", FbxNodeAttribute.eMesh: "Mesh", FbxNodeAttribute.eNurbs: "Nurbs", FbxNodeAttribute.ePatch: "Patch", FbxNodeAttribute.eCamera: "Camera" }
    result = node.GetNodeAttribute()
    if result:
        node_kind = result.GetAttributeType()
        if kinds.has_key(node_kind):
            print "Is A ", kinds[node_kind]
        else:
            print "Unknown Type ", node_kind
    if (result) and (node_kind == FbxNodeAttribute.eMesh):
        process_mesh(node)
    return result

In [85]:
def walk_nodes_aux(node,fn):
    for i in range(node.GetChildCount()):
        print node.GetChild(i).GetName()
        fn(node.GetChild(i))
        walk_nodes_aux(node.GetChild(i), fn)
        
def walk_nodes(scene, fn):
    root = scene.GetRootNode()
    fn(root)
    walk_nodes_aux(root, fn)

In [97]:
walk_nodes(root_node.GetScene(), classify_node)

Character
Is A  Mesh
Mesh has 2 layers
['__doc__', '__init__', '__module__', 'elements', 'get_element', 'get_polygon_element', 'get_vertex_element', 'indexed', 'layer', 'mesh', 'valid']
Mesh has 1 materials
Layer 0 
Layer has 1 uvsets 
['__add__', '__class__', '__contains__', '__delattr__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__getslice__', '__gt__', '__hash__', '__init__', '__le__', '__len__', '__lt__', '__mod__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmod__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_formatter_field_name_split', '_formatter_parser', 'capitalize', 'center', 'count', 'decode', 'encode', 'endswith', 'expandtabs', 'find', 'format', 'index', 'isalnum', 'isalpha', 'isdecimal', 'isdigit', 'islower', 'isnumeric', 'isspace', 'istitle', 'isupper', 'join', 'ljust', 'lower', 'lstrip', 'partition', 'replace', 'rfind', 'rindex', 'rjust', 'rpartitio